In [ ]:
#1-  uploading file to a dataframe
# uploading a csv file 

import pandas as pd # we need this package to manipulate data structues for data analysis and statistics
df = pd.read_csv("MulticlassTextDataset.csv", delimiter=',', dtype='unicode') 
#filename : give the full path to your file. if you file is in the same directory than your actual code, so just put the name of the file
#delimiter: use the delimiter used in your file, for example here mine was ","

# uploading a json file
#import json
#with open('TweetAnalysis.json') as json_file:
    #data = json.load(json_file)
# convert the json file to dataframe we can later manipulate
#df = DataFrame(Data, columns= ["tweet_id", "sentiment", "author", "content"])
#columns: are the names of the columns that you have in your json file

In [ ]:
print(len(df['Consumer complaint narrative']))

In [ ]:
#2-  visualisation of the data
#give you the information about the file : number of rows and columns, data type of each and the memory storage
df.info()
#give you an overview of the 200 first rows
df.head(200)
#deleting the rows with no consumer complanit naratif 
#you totaly must delete the unnecessary rows so you won't have data type issues
df = df[pd.notnull(df['Consumer complaint narrative'])]
print(len(df['Consumer complaint narrative']))
#give you the nomber of token (words, digit, ponctuation, ... ) splited using the blnak space
df['Consumer complaint narrative'].apply(lambda x: len(x.split(' '))).sum()

In [ ]:
#3-  text pre-processing
############# a- cleaning the data
#text before preprocessing 
print(df['Consumer complaint narrative'][12000])
#Delete the rows that have nothing in the relevant columns we gonna use : consumer complaint, company public response , product and issue
df = df[pd.notnull(df['Consumer complaint narrative'])]
df = df[pd.notnull(df['Product'])]
df = df[pd.notnull(df['Issue'])]
df = df[pd.notnull(df['Company public response'])]
print(len(df['Consumer complaint narrative']))
import nltk #world's most relevant package to text processing
import re # package to regular expression to find , substract a substring from the text
from nltk.corpus import stopwords #package of predefined stop words available in nltk
from stop_words import get_stop_words # specified package of predefined stop words
from nltk.tokenize import sent_tokenize, word_tokenize # split the text into words/tokens

########################################################################################################
def cleaning_data (text) :
    #this function process to the cleaning by using regular expression
    #lover the text
    text= text.lower()
    #links
    text=re.sub('https://\S+','lien',text)
    text=re.sub('http\S+\s','lien',text)
    #email
    text=re.sub('\w+@\w+','',text)
    #ponctualtion
    text=re.sub('[%s]' % re.escape("""!"#/\$%&()*+',-.:;<=>?[]^_`{|}~°•"""), ' ', text)
    text=re.sub('[%s]' % re.escape('""'), ' ', text)
    text=re.sub('[%s]' % re.escape("'"), ' ', text)
    # remove special characters and digits
    text=re.sub("(\d)+"," ",text) 
    #remove +2 blank spaces
    text = re.sub('\s+', ' ', text)
    #words with 1 to 2 letters
    text= re.sub(r'\W*\b\w{1,2}\b', ' ', text)
    #after visualiting the data we find that xxxx and xxx was used a lot , so we gonna just remove it 
    text= re.sub('xxx', ' ', text)
    text= re.sub('xxxx', ' ', text)
    #stop words
    stopwords = get_stop_words('en') # stopwords from stopwords packages
    text = ' '.join(word for word in text.split() if word not in (stopwords))
                    
    ####or you can use the nltk collection of stopwords
    #set(stopwords.words('english'))
    #stop_words = set(stopwords.words('english'))  
    #text = ' '.join(word for word in text.split() if word not in (stop_words)
    #non-relevant words               
    nrelevant=['did']
    words=word_tokenize(text)
    for word in words:
        if word.lower() in nrelevant:
            text=text.replace(word,' ')
    #remove +2 blank spaces
    text = re.sub('\s+', ' ', text)
    return text
########################################################################################################
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(lambda x:cleaning_data(x))   
print(df['Consumer complaint narrative'][12000])
print(len(df['Consumer complaint narrative']))

In [ ]:
#cleaning also the Company public response text data
df['Company public response'] = df['Company public response'].apply(lambda x:cleaning_data(x)) 
print(len(df['Consumer complaint narrative']))
#after preprocessing token count in the dataset (consumer complaint narrative)
df['Consumer complaint narrative'].apply(lambda x: len(x.split(' '))).sum()

In [ ]:
#3-  text pre-processing
############# b- data transformation
freq = pd.Series(' '.join(df['Consumer complaint narrative']).split()).value_counts()[-50:] #select the less frequent words (with less than 55 apparition in the whole dataset) in the dataset
freq = list(freq.index)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
print(len(df['Consumer complaint narrative']))
df['Consumer complaint narrative'].apply(lambda x: len(x.split(' '))).sum()


In this project, with this data, we are not gonna use the data transformation below. It's just an overview of what you can do in order to simplfy and unify you data in order to increase the accuracy of the futur model (if yiu have a looot of noise in you data for example, or you want to create a corpus/dictionnary)

In [ ]:
##########################
from textblob import TextBlob
from nltk.tokenize import sent_tokenize, word_tokenize

#Spelling correction
df['Consumer complaint narrative'][:5].apply(lambda x: str(TextBlob(x).correct()))
print(df['Consumer complaint narrative'][12000])

#tokenization using ngram
words=TextBlob(df['Consumer complaint narrative'][12000]).words #with textblob
words=word_tokenize(df['Consumer complaint narrative'][12000])  #with nltk
tokens=nltk.ngrams(words,3)
ngram_list=[]
for ngram in tokens:
    lowered_ngram_tokens = map(lambda token: token.lower(), ngram)
    for token in lowered_ngram_tokens:
        ngram_list.append(' '.join(ngram))
print( ngram_list )

#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['Consumer complaint narrative'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
#or with nltk

#Porter Stemming
from nltk.stem import PorterStemmer 
lancaster=LancasterStemmer()
def stemSentence(text):
    porter = PorterStemmer() #create an object of class PorterStemmer
    token_words=word_tokenize(text)
    #token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(lambda x:stemSentence(x)) 
###########################

#Lancaster Stemming
from nltk.stem import LancasterStemmer
def stemSentence(text):
    lancaster=LancasterStemmer() #create an object of class LancasterStemmer
    token_words=word_tokenize(text)
    #token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(lancaster.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(lambda x:stemSentence(x)) 
###########################
#for non english text I suggest to you to use the snowball stemmer 
#also available in english
from nltk.stem.snowball import SnowballStemmer
#frenchStemmer=SnowballStemmer("french")  #example with a frenchstemming
#Snowball Stemming
def stemSentence(text):
    englishStemmer=SnowballStemmer("english") #create an object of class SnowballStemmer
    token_words=word_tokenize(text)
    #token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(englishStemmer.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(lambda x:stemSentence(x)) 


#lemmatisation
from textblob import Word
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df['Consumer complaint narrative'].head()
#or with nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(lambda x: " ".join([wordnet_lemmatizer.lemmatize(word) for word in x.split()]))

In [ ]:
#4-  saving the data in a new file 
df.to_csv("after_preprocessing.csv", sep=',', encoding='utf-8', index=False, header=True)

In [ ]:
#5-  extraction of relevant information
#         URL EXTRACTION         
from urlextract import URLExtract
extractor = URLExtract()
urls = extractor.find_urls("i have an issue while opening the link http://www.site.com/link1 and also the url to the form : www.site.com/form")
print(urls)  #return a list of url extracted from the string below
###########################################
#         Named Entity Recognition : NER : detecte places, dates, organization's name, money references, ...
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
text_processing = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
#article = nlp(ny_bb)
len(text_processing.ents)
labels = [x.label_ for x in text_processing.ents]
print(text_processing.ents)# the entity detected from the text 
print(labels) #the label of each entity example: org, place, date, money , ...


In [ ]:
#6 spliting the data set to a 85% training data and 15% test data
print(len(df['Consumer complaint narrative']))
from sklearn.model_selection import train_test_split # split the data to train and test set
X = df['Consumer complaint narrative']
y = df['Product']
z= df['Issue']
# spliting the data set to a 85% training data and 15% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 100)
X_train

####For the KNN model we need to process differently 
#preparation des données pour l'entrainement et le test
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords=get_stop_words('en')
vectorizer = TfidfVectorizer(stopwords) 
KNN_X = vectorizer.fit_transform(df['Consumer complaint narrative']) #we vectorize the data outside the model description
KNN_y = df['Product']
KNN_X_train, KNN_X_test, KNN_y_train, KNN_y_test = train_test_split(KNN_X, KNN_y, test_size=0.20, random_state = 20)

In [ ]:
#7-  Construction of the model : using tf-idf to the vectorization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
############# a-  Naive Bayes
from sklearn.naive_bayes import MultinomialNB 
ngram_range=(1,1)  #we gonna split the data into a unigram (word by word) before the vectorisation, to count It presence in the dataset
nb = Pipeline([('vect', CountVectorizer(ngram_range=ngram_range)),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB(alpha=1,n_jobs=-1)),
              ])
%time

In [ ]:
############# b-  Logistic regression
from sklearn.linear_model import LogisticRegression #one vs all
logreg = Pipeline([('vect', CountVectorizer(ngram_range=ngram_range)),
                    ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression( C=10,penalty ='l1', n_jobs=-1)), #n_jobs=-1 means that you will use all the process in your computer to do this job
               ])
%time

In [ ]:
############# c-  linear SVM
from sklearn import svm # one vs one
SVM = Pipeline([('vect', CountVectorizer(ngram_range=ngram_range)),
                ('tfidf', TfidfTransformer()),
                ('clf', svm.SVC(kernel='linear', C = 1.0,n_jobs=-1)),
               ])
%time

In [ ]:
############# d-  SVM with SGD
from sklearn.linear_model import SGDClassifier #one vs rest
sgd = Pipeline([('vect', CountVectorizer(ngram_range=ngram_range)),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='log',n_jobs=-1, penalty='l2',alpha=1e-6, random_state = 200, max_iter =150)),
               ])
%time

In [ ]:
############# e-  Random forest
from sklearn.ensemble import RandomForestClassifier
randomf= Pipeline([('vect', CountVectorizer(ngram_range=ngram_range)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', RandomForestClassifier(n_estimators=20, criterion='entropy',n_jobs=-1))
                  ])
%time
#n_estimators: number of trees, by default is 10
#creterion: measure the quality of a split, by default it "gini" impurity

In [ ]:
############# f-  K nearest nighbors
from sklearn.neighbors import KNeighborsClassifier
# Clustering the document with KNN classifier
#we use the metric of distance to weights the neighbors
#we consider the 4 nearest neighbors to check in which cluster/class the data lean on
modelknn = KNeighborsClassifier(n_neighbors=4, n_jobs=-1, weights='distance')

In [ ]:
#7-  Construction of the model : by selecting the K best features in the corpus to vectorize the text, using the chi2 method
#Example on the naive bayes model
from sklearn.feature_selection import SelectKBest, chi2
#Naive Bayes classifier for multinomial models
ngram_range=(1,1)
nb = Pipeline([('vect', CountVectorizer(ngram_range=ngram_range)),
               ('chi',  SelectKBest(chi2, k=20000)), #we are going to select the 20000 best features (most referenced in the whole dataset) to describe and so vectorize the text
               ('clf', MultinomialNB(alpha=1)), 
              ])
%time

In [ ]:
#8-  training the model
############# a-  Naive Bayes
nb.fit(X_train, y_train)
%time

In [ ]:
#9-  test & evaluation ofthe model
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
#my_tag represent the label that specify every single group/class in your dataset. Here for example is the type of the product
my_tags=["Bank account or service","Checking or savings account","Consumer Loan","Credit card","Credit card or prepaid card","Credit reporting","Credit reporting, credit repair services, or other personal consumer reports","Debt collection","Money transfer, virtual currency, or money service","Money transfers","Mortgage","Other financial service","Payday loan","Payday loan, title loan, or personal loan","Prepaid card","Student loan","Vehicle loan or lease","Virtual currency"]
############# a-  Naive Bayes
y_pred = nb.predict(X_test)
%time
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))


In [ ]:
#8-  training the model
############# b-  Logistic regression
logreg.fit(X_train, y_train)
%time

In [ ]:
#9-  test & evaluation ofthe model
############# b-  Logistic regression
y_pred = logreg.predict(X_test)
%time
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
#8-  training the model
############# c-  linear SVM
SVM.fit(X_train, y_train)
%time

In [ ]:
#9-  test & evaluation ofthe model
############# c-  linear SVM
y_pred = SVM.predict(X_test)
%time
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
#8-  training the model
############# d-  SVM with SGD
sgd.fit(X_train, y_train)
%time

In [ ]:
#9-  test & evaluation ofthe model
############# d-  SVM with SGD
y_pred = sgd.predict(X_test)
%time
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
#8-  training the model
############# e-  Random forest
randomf.fit(X_train, y_train)
%time

In [ ]:
#9-  test & evaluation ofthe model
############# e-  Random forest 
y_pred = randomf.predict(X_test)
%time
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
#8-  training the model
############# f-  K nearest nighbors
modelknn.fit(KNN_X_train,KNN_y_train)
%time

In [ ]:
#9-  test & evaluation ofthe model
############# f-  K nearest nighbors
KNN_y_pred = modelknn.predict(KNN_X_test) 
%time
print('accuracy %s' % accuracy_score(KNN_y_pred, KNN_y_test))
print(classification_report(KNN_y_test, KNN_y_pred))

In [ ]:
#10-  saving the model

from sklearn import model_selection
import pickle
with open('svm_classifier', 'wb') as picklefile:  
    pickle.dump(SVM,picklefile)

In [ ]:
#11-  tuning the parameters of the model

############ a- NB- LR- RF- SVM
# the example below for the svm classifier, can apply to any of the other classifier, for any of their parameters
# gridsearch will build, test and evaluate each model from the other and will give up the appropriate one with the specific parameters

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2),(1,3),(2,2)],
               'tfidf__use_idf': (True, False),
              'clf__C': (1e1,1.0),
              
             }
gs_clf = GridSearchCV(SVM, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)
print(gs_clf.best_score_)
print(gs_clf.best_params_)

In [ ]:
#11- tuning the parameters of the model
############ B- KNN
#as the main parameter that import in the KNN model is the number of neighbors that we consider in the model
#the paremter tuning will be to find the best one with the best accuracy

error = []
# Calculating error for K values between 1 and 40
for i in range(1, 40):  #you can obviously change the range you wanna test 
    print(i)
    knn = KNeighborsClassifier(n_neighbors=i,n_jobs=-1, weights='distance') # we build the model , adding the other parameters
    knn.fit(KNN_X_train, KNN_y_train) # train it 
    KNN_pred_i = knn.predict(KNN_X_test) #test it
    error.append(np.mean(KNN_pred_i != KNN_y_test))


In [ ]:
# then we will use a graph to show the evolution of the error while the number of neighbors rise
plt.figure(figsize=(20, 20))  
plt.plot(range(1, 40), error, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('Error Rate K Value')  
plt.xlabel('K Value')  
plt.ylabel('Mean Error') 